In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import Counter
import csv, re, pickle

from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
import numpy as np
from keras.utils import to_categorical


Using TensorFlow backend.


In [2]:
data = pd.read_csv("totalReviewWithSuggestion.csv")
reviews = data['Text']
rate = data['Score']
labels = data['Suggestion']

In [76]:
# labels = np.array([1 if int(each)==1 or int(each)==2 else 0 for each in labels])

In [6]:
#cleaning dataset
words=[]
all_text = ''

for t in range (len(reviews)):
    text = reviews[t]
    text = text.replace('\u200c',' ')
    text = re.sub(r'[^a-zA-Z0-9آ-ی۰-۹ ]', ' ', text)
    all_text += text
    all_text += ' '
    wordsInText = text.split()
    for word in wordsInText:
        if word != ' ' or word != '':
            words.append(word)

In [78]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

with open("mySavedDict.txt", "wb") as myFile:
    pickle.dump(vocab_to_int, myFile)

'''
with open("mySavedDict.txt", "rb") as myFile:
    myNewPulledInDictionary = pickle.load(myFile)
'''
#print myNewPulledInDictionary

'\nwith open("mySavedDict.txt", "rb") as myFile:\n    myNewPulledInDictionary = pickle.load(myFile)\n'

In [79]:
reviews_ints = []
for each in reviews:
    #print (each)
    each = each.replace('\u200c',' ')
    each = re.sub(r'[^a-zA-Z0-9آ-ی۰-۹ ]', ' ', each)
    reviews_ints.append([vocab_to_int[word] for word in each.split()])


'''
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))
'''

'\nreview_lens = Counter([len(x) for x in reviews_ints])\nprint("Zero-length reviews: {}".format(review_lens[0]))\nprint("Maximum review length: {}".format(max(review_lens)))\n'

In [80]:
lstm_size = 256
lstm_layers = 1
batch_size = 200
learning_rate = 0.001

data_dim = 16
timesteps = 25
num_classes = 2

n_words = len(vocab)
print (n_words)

12288


In [81]:
seq_len = 100
features = np.zeros((len(reviews), seq_len), dtype=int)
for i, row in enumerate(reviews_ints):
#     print (i , row)
#     print ('****')
    features[i, -len(row):] = np.array(row)[:seq_len]

split_frac = 0.8
split_idx = int(len(features)*split_frac)
x_train, x_val = features[:split_idx], features[split_idx:]
y_train, y_val = labels[:split_idx], labels[split_idx:]

test_idx = int(len(val_x)*0.5)
x_val, x_test = val_x[:test_idx], val_x[test_idx:]
y_val, y_test = val_y[:test_idx], val_y[test_idx:]


print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(x_train.shape), 
      "\nValidation set: \t{}".format(x_val.shape),
      "\nTest set: \t\t{}".format(x_test.shape))

			Feature Shapes:
Train set: 		(2608, 400) 
Validation set: 	(163, 400) 
Test set: 		(163, 400)


In [82]:
x_test = np.reshape(x_test, (163, timesteps,data_dim))
x_train = np.reshape(x_train, (2608,timesteps,data_dim))
x_val = np.reshape(x_val, (163,timesteps,data_dim)) 
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(x_train.shape), 
      "\nValidation set: \t{}".format(x_val.shape),
      "\nTest set: \t\t{}".format(x_test.shape))

			Feature Shapes:
Train set: 		(2608, 25, 16) 
Validation set: 	(163, 25, 16) 
Test set: 		(163, 25, 16)


In [83]:
y_val = to_categorical(y_val)
y_train = to_categorical(y_train)


In [84]:

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
# model.add(Flatten(input_shape=(1,32)))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_55 (LSTM)               (None, 25, 32)            6272      
_________________________________________________________________
lstm_56 (LSTM)               (None, 25, 32)            8320      
_________________________________________________________________
lstm_57 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total params: 22,978
Trainable params: 22,978
Non-trainable params: 0
_________________________________________________________________


In [85]:
model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))

Train on 2608 samples, validate on 163 samples
Epoch 1/5
2608/2608 [==============================] - 6s 2ms/step - loss: 0.4447 - acc: 0.8497 - val_loss: 0.4596 - val_acc: 0.8344
Epoch 2/5
2608/2608 [==============================] - 2s 664us/step - loss: 0.4195 - acc: 0.8535 - val_loss: 0.4548 - val_acc: 0.8344
Epoch 3/5
2608/2608 [==============================] - 2s 669us/step - loss: 0.4152 - acc: 0.8535 - val_loss: 0.4526 - val_acc: 0.8344
Epoch 4/5
2608/2608 [==============================] - 2s 668us/step - loss: 0.4155 - acc: 0.8535 - val_loss: 0.4546 - val_acc: 0.8344
Epoch 5/5
2608/2608 [==============================] - 2s 688us/step - loss: 0.4132 - acc: 0.8535 - val_loss: 0.4535 - val_acc: 0.8344


In [1]:
# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

embed_size = 500 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words+1, embed_size), -1, 1))
    #print ('len input   :   ', inputs_)
    embed = tf.nn.embedding_lookup(embedding, inputs_)
    #print (embed[46421])

with graph.as_default():
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)

with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)


with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            print (embed[46421])
            feed = {inputs_: x, labels_: y[:, None], keep_prob: 0.5, initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,labels_: y[:, None],keep_prob: 1,initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

#46421

NameError: name 'tf' is not defined